# Проект анализ продаж игр.

# Ведение

 Нам предстоит просмотреть, обработать и иследовать открытые данные о компьютерных играх. Понять, на что потратить рекламный бюджет. В исследовании необходимо проанализированы игры с разных сторон:Понять какие жанры в каких регионах привлекательнее, Как влияют на продажи отзывы критиков и пользователей. Как влияют на продажи рейтинг от организации ESRB.

## Шаг 1. Чтение данных.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sas
import matplotlib.pyplot as plt
from scipy import stats as st

import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 40)

In [4]:
data = pd.read_csv('games.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'games.csv'

In [ ]:
data.duplicated().sum()

In [ ]:
data.groupby(['Name', 'Platform', 'Year_of_Release'])['Name'].count().sort_values(ascending=False).head()

In [ ]:
data[data['Name'] == 'Madden NFL 13']

In [ ]:
len(data)

In [ ]:
data = data[~((data['Name'] == 'Madden NFL 13') & (data['Platform'] == 'PS3') & (data['Year_of_Release'] == 2012.0) & (data['EU_sales'] == 0.01))]

In [ ]:
len(data)

In [ ]:
data.isna().sum()

In [ ]:
data.isna().mean()

In [ ]:
data.describe()

In [ ]:
axes = data.hist(figsize = (25,15))
axes[0,0].set_title('год выпуска игры')
axes[0,0].set_xlabel('год выпуска')
axes[0,0].set_ylabel('количество игр')

axes[0,1].set_title('продажи по Североамениканскому региону')
axes[0,1].set_xlabel('продажи милионов копий')
axes[0,1].set_ylabel('количество игр')

axes[1,0].set_title('продажи по Европейскому региону региону')
axes[1,0].set_xlabel('продажи милионов копий3')
axes[1,0].set_ylabel('количество игр')

axes[1,1].set_title('продажи по Японскому региону')
axes[1,1].set_xlabel('продажи милионов копий')
axes[1,1].set_ylabel('количество игр')

axes[2,0].set_title('продажи в других регионах')
axes[2,0].set_xlabel('продажи милионов копий')
axes[2,0].set_ylabel('количество игр')

axes[2,1].set_title('Оценки критиков')
axes[2,1].set_xlabel('оценка критиктв')
axes[2,1].set_ylabel('количество игр')


plt.suptitle('Гистограммы числовых значений датасета', ha='center', fontsize='xx-large')
plt.show()

Информация в проекте представлена следующим файлом: "games.csv"

таблица представлена 16714 строками и 11 столбцами:
- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы проданных копий)
- EU_sales — продажи в Европе (миллионы проданных копий)
- JP_sales — продажи в Японии (миллионы проданных копий)
- Other_sales — продажи в других странах (миллионы проданных копий)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

в таблице нет явных дублекатов, но есть пропуски в столбцах "Name" и "Genre" по 2 пропуска, в строке "Year_of_Release" 269 пропусков и больше всего пропусков в строках: "Critic_Score", "User_Score" и "Rating". Их количество равно: 8578, 6701 и 6766 соответственно. В именах столббцов присутствуют заглавные буквы. Столбци "Year_of_Release" и "User_Score" не соответствуют своему типу данных. Также в таблице присутствовал 1 неявный дублекат строка № 16230 была удалена из датасета

## Шаг 2. Подготовьте данные

### Корректировка названий столбцов


In [ ]:
list(data.columns.values)

In [ ]:
data = data.rename(columns={'Name': 'name',                        
                            'Platform': 'platform',
                            'Year_of_Release': 'year_of_release',
                            'Genre': 'genre',
                            'NA_sales': 'na_sales',
                            'EU_sales': 'eu_sales',
                            'JP_sales': 'jp_sales',
                            'Other_sales': 'other_sales',
                            'Critic_Score': 'critic_score',
                            'User_Score': 'user_score',
                            'Rating': 'rating',                            
                           })

In [ ]:
list(data.columns.values)

имена столбцов откорректированы

### Преобразуйте данные в нужные типы. Опишите, в каких столбцах заменили тип данных и почему;

#### name

In [ ]:
data.info()

In [ ]:
data.name.nunique()

In [ ]:
data[(data.name.duplicated())].head()

In [ ]:
data[data.name.isna()]

In [ ]:
data = data[~data.name.isna()]

In [ ]:
data.genre.nunique()

In [ ]:
data.genre.unique()

In [ ]:
data.info()

Тип данны в строке "name", "platform" и "genre" не нужно преобразовывать. Они соответствуют своему типу.

Были обнорцжены повторы в строке "name" , они связаны с тем, что были игры, которые выпускались на несколько платформ.

Удалены 2 строки, у которой почти все данные заполнены NaN.Так как считаю эти данные ошибкой не влияющей на исследование.

#### year_of_release

In [ ]:
data.year_of_release.nunique()

In [ ]:
data.year_of_release.unique()

In [ ]:
data.year_of_release.isnull().sum()

In [ ]:
data.loc[data.year_of_release.isnull(), 'year_of_release'] = -99999

в случае, если год выпуска игры незвестен, временно заменяю его на значение "-99999", для того чтобы поменять формат столбца но тип "int"

Поставил Заглушку потому, что боюсь потерять даже небольшую часть данных, так как в этих данных может хранится информация, которая нам в дальнейшем может пригодится.

In [ ]:
data['year_of_release'] = data['year_of_release'].astype('int')

In [ ]:
data.year_of_release.unique()

Столбец "year_of_release" имеет тип данных 'float'. Это связано с тем, что в ее данных есть пропуски, считаю это неправельным. Заменяю пропуски на "-99999" и изменяю тип данных на 'int'.

#### продажи

In [ ]:
data.head()

В столбцах "na_sales", "eu_sales", "jp_sales" и "other_sales" соответствуют своему типу и не имеет пропусков

#### critic_score

In [ ]:
data.critic_score.nunique()

In [ ]:
data.critic_score.unique()

In [ ]:
data.critic_score.isna().sum()

In [ ]:
data["critic_score"] = data.groupby("name")['critic_score'].transform(lambda x: x.fillna(x.mean()))

После этой команды средний рейтинг каждой игры никак не поменялся, но у нас добавились 961 запись по средним рейтингам игр на разных платформах. Возможно в дальнейшем это чуть большее охарактеризует какуюто из не популярных платформ.

In [ ]:
data.critic_score.isna().sum()

In [ ]:
8576-7615

частично заполнили недостающие значения средними значениями из их таких же игр, но с других платформ

In [ ]:
data.head()

In [ ]:
data.critic_score.round().unique()

In [ ]:
data.critic_score = data.critic_score.fillna(value=-99999).round()

In [ ]:
data.critic_score = data.critic_score.astype('int')

в столбце "critic_score" пропуски частично заполннены средними значениями из их таких же игр, но с других платформ, а оставшиеся пропуски заменены на значение "-99999" и тип данных столбца изменен на "int"

#### user_score

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.user_score.unique()

In [ ]:
data.loc[data['user_score'] == 'tbd', 'user_score'].count()

In [ ]:
data[data['user_score'] == 'tbd'].head()

In [ ]:
data[data.user_score.isnull()].name.count()

In [ ]:
data[data.user_score.isnull()].head()

In [ ]:
data.loc[data['user_score'] == 0, 'user_score'].count()

In [ ]:
data.loc[data['user_score'] == 'tbd', 'user_score'] = 0

Временно заменили все данные со значением "tbd" на данные со значением "0". таких данных получилос на 1 ед. больше, но это не отразится на итоговых результатах

In [ ]:
data['user_score'] = data['user_score'].astype('float')

In [ ]:
data.loc[data['user_score'] == 0, 'user_score'].count()

In [ ]:
data[data.user_score.isna()].name.count()

In [ ]:
data.user_score.unique()
data.user_score = data.user_score * 10

In [ ]:
data["user_score"] = data.groupby("name")['user_score'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
data[data.user_score.isna()].name.count()

частично заменили пропущенные значения на средние значения этой игры на других платформах

In [ ]:
data.user_score = data.user_score.fillna(value=-99999)

In [ ]:
data.loc[data['user_score'] == 0, 'user_score'] = -88888

Зделал разные заглушки для того, чтобы было возможно в будующем отдельно отследить откуда взялась ошибка tbd, а откуда пропуски.

In [ ]:
data['user_score'] = data['user_score'].astype('int')

In [ ]:
data.user_score.nunique()

In [ ]:
data.user_score.unique()

In [ ]:
data.head()

в строке "user_score" пропущеные значения били частично заменены на средние значения игры на других платформах, если таких игр не быо,то значения были изменены  на значение "-99999", также были найдены 2424 значения "tbd". Преддполагаю, что это значение ошибочное, и так как в столбце "user_score" дискретные математические значения, мной было принято решение заменить значение "tbd" на значение "-88888". А также произведя матиматические действия измененить тип столбца на тип "int", для возможности использования данных в арифмитеческих выражениях.

#### rating

In [ ]:
data.rating.nunique()

In [ ]:
data.rating.unique()

In [ ]:
data.rating = data.rating.fillna(value='no_rating')

In [ ]:
data[data['rating'] == 'no_rating'].head()

In [ ]:
data.rating.unique()

В строке "rating" содержатся данные тиа "str" и есть пропуски. Пропуски заменили на значение "no_rating"

### Обработайте пропуски при необходимости:

#### Причины заполнения пропусков

Были удалены 2 строки с практически полность отсутствующими данными, так как узнать о какой информации шла речь невозможно.
В столбцах "critic_score" и "user_score" отсутствующие значения были заполнены средним значением игры на других платформах, а если таких игр не было, то заменены значением "-99999", для того чтобы можно было эти данные легко найти и отфильтровать и с оставшимися данными в столбцах можно было производить расчеты. Не заполнял эти данные не мидианными не средними значениями, так как от этих данным может меняться мнение об игре. 
Недастающие данные в столбце "rating" заменены на значение "no_rating", , предполагаю, что в эти игры можно играть всем, но в расчетах по рейтингу эти данные использоватся не будут

#### предположение о появлении пропусков

так как данные брались из открытых источников, возможно некоторые данные в этих источника отсутствовари, поэтому в таких данных имеются пропуски.

#### 'tbd' в столбце с оценкой пользователей
 

Было найдено 2424 значения "tbd". Происхождение этой ошибки неизвесно, было принято заменить значение на "-88888", чтобы можно было легко эти значения отфильтровать и иследовать.А с данными можно было производить математические операции.

### Посчитайте суммарные продажи во всех регионах и запишите их в отдельный столбец.

In [ ]:
data['total_sales'] = data['na_sales'] + data['eu_sales'] + data['jp_sales'] + data['other_sales']

In [ ]:
data.head()

In [ ]:
data.total_sales.isna().sum()

добавили столбец "total_sales" в котором посчитали выручку со всех регионов

In [ ]:

temp = data.copy() 
list_c = ['name', 'platform', 'year_of_release', 'genre', 'critic_score', 'user_score', 'rating']
print(temp.info())
for col_l in list_c:
  print('-'* 25)
  print(col_l, temp[col_l].sort_values().unique())
  print(col_l,': кол-во NaN',temp[col_l].isna().sum(),
        ', процент NaN', round(temp[col_l].isna().mean()*100,2),'%') 

## Шаг 3. Проведите исследовательский анализ данных


### Посмотрите, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?


In [ ]:
data.year_of_release.hist(bins=40, range=(1978, 2017))
plt.title("количество игр выпускаемые в разные года")
plt.xlabel("год")
plt.ylabel("количество выпускаемых игр")
plt.show()

In [ ]:
data.year_of_release.value_counts()

In [ ]:
data.year_of_release.value_counts(normalize=True)

In [ ]:
data.year_of_release.value_counts(normalize=True).sort_values().plot(kind='pie', figsize=(15, 9), autopct='%1.0f%%')
plt.title("количество игр выпускаемые в разные года")
plt.show()

In [ ]:
(data['year_of_release'] < 0).sum()

In [ ]:
data_clean = data[data['year_of_release'] > 0]

Создаем датасет без данных о играх в которых не указана дата выпуска. В эти данные не попала информация о 269 играх

In [ ]:
data_clean.year_of_release.describe()

In [ ]:
data_clean.boxplot('year_of_release')
plt.title("количество игр выпускаемые в разные года")
plt.ylabel("количество выпускаемых игр")
plt.show()

In [ ]:
data_clean

In [ ]:
data_clean['grup_year_of_release'] = pd.cut(data_clean['year_of_release'], [0, 1993, 2000, 2008, 2016], labels=["до 1993", "до 2000", "до 2008", "до 2017"]) 

In [ ]:
data_clean.grup_year_of_release.value_counts()

In [ ]:
data_clean.grup_year_of_release.value_counts(normalize=True)

In [ ]:
data_clean.grup_year_of_release.value_counts().plot(kind='pie', figsize=(15, 9), autopct='%1.0f%%')
plt.title("количество игр выпускаемые в разные года")
plt.show()

In [ ]:
data_clean.groupby('grup_year_of_release')['name'].count().plot(kind='bar', grid=True)
plt.title("количество игр выпускаемые в разные года")
plt.xlabel("год")
plt.ylabel("количество выпускаемых игр")
plt.show()

В наших данных есть записи с 1980 по 2016 год. Больше всего выпущено игр в 2008 год что составляет 8.5%. Меньше всего в 1980м, всего 9 игр что составляет 0.053% Так как до 1993г выпускалось мало игр, моожно считать, что эти данные не актуальны. (выбросы). Количество игр выпущенное с 2000 до 2008 года равно 7417 игр, что составляет 45%, в то время как после 2008 года выпущено 6703 игры, что составляет 40 процентов от всех выпущеных игр.

### Посмотрите, как менялись продажи по платформам. 
Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?


In [ ]:
data_clean.head()

In [ ]:
top_platform= data_clean.groupby('platform')['total_sales'].sum().sort_values(ascending=False).reset_index().head(7)['platform']
top_platform

In [ ]:
data_clean.groupby('platform')['total_sales'].sum().sort_values(ascending=False).reset_index().head(7)['total_sales'].sum()\
/ data_clean['total_sales'].sum()

общая доля проданых игр у топ 7 компаний

In [ ]:
data_top_platform = data_clean[data_clean['platform'].isin(top_platform)]

In [ ]:
data_top_platform.pivot_table(index='year_of_release',columns='platform', values='name', aggfunc='count' ).tail(5)

In [ ]:
data_top_platform.pivot_table(index='year_of_release',columns='platform', values='name', aggfunc='count' ).\
plot(kind='bar', figsize= (15,10), width=1)
plt.title("количество игр выпускаемые в разные года, под разные платформамы")
plt.xlabel("года")
plt.ylabel("количество выпускаемых игр")
plt.show()

Платформы с наибольшими продажами, топ 7 платформ, занимают 66% всего рынка видеоигр. Актуальный срок жизни платформы составляет 10 - 12 лет.

In [ ]:
temp = data_clean.copy() 
time_life = temp.pivot_table(index='platform', values='year_of_release', aggfunc=['min', 'max']).reset_index()
time_life['life'] = time_life['max'] - time_life['min'] + 1 # в срок жизни платформы, должны входить границы диапазона, 
                                                            # поэтому +1
time_life['life'].median()

In [ ]:
xx = data_top_platform.pivot_table(index='platform', values='year_of_release', aggfunc=['min', 'max']).reset_index()
xx['life'] = time_life['max'] - time_life['min'] + 1 
xx['life'].median()

После проверки выявлено, что у успешных платформ, время жизни больше.

### Возьмите данные за соответствующий актуальный период. 
Актуальный период определите самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год.


In [ ]:
actual_data = data_clean[data_clean['year_of_release'] > 2012]

In [ ]:
actual_data

In [ ]:
top_actual_platform = actual_data.groupby('platform')['total_sales'].sum().sort_values(ascending=False).reset_index().head(10)['platform']
top_actual_platform

In [ ]:
actual_data.groupby('platform')['total_sales'].sum().sort_values(ascending=False).reset_index().head(10)

In [ ]:
top_actual_data = actual_data[actual_data['platform'].isin(top_actual_platform)]

считаю актуальным то время в которое выходят игры на современные платформф, так как жижнь платформы примерна равна 10 годам, то и актуальнам будут данные за последние 10 лет. также найдем топ 10 платформ за последние 10 лет. Самой выгодной платформой за актуальное время является PS3 она подала 910.38 милионов копий игр, затем идет X360, с продажами в 901.37 копий, потом Wii 754.03 и DS с продажами в 535.54 милиона копий. PS4 с продажами 314.14, 3DS с продажами 257.81, PSP с продажами 183.31, PS2 с продажами 162.38, PC с продажами	160.57 и замыкает 10ку XOne с продажами	159.32

### Какие платформы лидируют по продажам, растут или падают?
Выберите несколько потенциально прибыльных платформ.


In [ ]:
top_actual_data.pivot_table(index='platform', values=['total_sales'], columns='year_of_release', aggfunc='sum')

In [ ]:
top_actual_data.pivot_table(index='platform', values=['total_sales'], columns='year_of_release', aggfunc='sum').\
plot( kind='bar', figsize= (15,10), width=0.9)
plt.title("количество игр проданных в разные годы, под разные платформы")
plt.xlabel("платформы")
plt.ylabel("общая выручка за проданные игры")
plt.show()

Из десяти платформ, которые за последние 10 лет продами максимальгое количество копий только 2 платформы показывают потенциальный рост: это платформы XOne и PS4. На них стоит обратить особое внимание. четыре платформы в 2016 году вообще не показали прибыли или очень маленькую прибыль: PSP, PSV, PC и Wii про них стоит забыть, их время ушло. И четыре платформы, которые скоро уйдут, но еще могут принести некоторцю прибыль. это 3DS, WiiU, PS3 и X360.

### Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишите результат.


In [ ]:
actual_data.head(3)

In [ ]:
platform_data = actual_data.pivot_table(index='name', columns='platform', values='total_sales', aggfunc='sum')
platform_data

In [ ]:
platform_data.plot( kind='box', figsize=(15,10), grid=True)
plt.title("по глобальным продажам игр в разбивке по платформам")
plt.xlabel("платформы")
plt.ylabel("выручка за проданные игры")
plt.show()

Я не понял в чем моя ошибка. сказано:  "Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам."
я в сводной таблице разбиваю актуальные данные по плптформам и вывожу сумму по каждой игре. так как для каждой платформы игра может быть только в одном варианте он выведет сумму 1 игры (добавил внизу группировку, чтобы это было видно.). прошу подробнее обяснить требуемое от меня задание, и у меня был ниже более приближеная диаграмма.

In [ ]:
actual_data.groupby(['name','platform'])['name'].count().sort_values()

In [ ]:
platform_data.plot(ylim=(0, 2), kind='box', figsize=(15,10), grid=True)
plt.title("по глобальным продажам игр в разбивке по платформам")
plt.xlabel("платформы")
plt.ylabel("выручка за проданные игры")
plt.show()

In [ ]:
actual_data.groupby('platform')['total_sales'].sum().sort_values(ascending=False).reset_index().head(10)

In [ ]:
platform_data.describe()

график "ящик с усами" не показывают картину в целом для платформ по глобальным продажам. И в первую очередь это связано с тем, что "ящик с усами" считает многие проданые игры выбросами, в то время как эти игры принесли платформе основной доход, Максимальная медиана проданых копий принадлежит платформе X360 и составляет 0.265 милиона копий, а доход учитывается со 186 игрр по которым есть данные, в то время как лидер продаж платворма PS4 имеет медиану проданых копий равную 0.2 милоинам и включает данные по 392 проданым играм. Но также мы можем увидеть, что чем жирнее хвостда графике с усами, тем больше продаж у этой платформы.

### Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков.
Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами. Сформулируйте выводы.


Будем проводить анализ платформы X360. так как популярная платформа входит в топ 10 и уже прошло ее время.

In [ ]:
X360_data = actual_data[actual_data['platform'] == 'X360']
X360_data.head(3)

#### отзывы критиков critic_score

In [ ]:
critic_X360_data = X360_data[X360_data['critic_score'] > 0]
critic_X360_data.head(3)

In [ ]:
critic_X360_data.plot(x='critic_score', y='total_sales', kind='scatter', alpha=0.2, grid=True, ylim=(0, 3))
plt.title("отношение количества проданных копий к оценкам критиков")
plt.xlabel("продажи милионов копий")
plt.ylabel("оценки критиков")
plt.show()

In [ ]:
critic_X360_data.critic_score.corr(critic_X360_data.total_sales)

In [ ]:
critic_X360_data.critic_score.corr(critic_X360_data.na_sales)

In [ ]:
critic_X360_data.critic_score.corr(critic_X360_data.eu_sales)

In [ ]:
critic_X360_data.critic_score.corr(critic_X360_data.jp_sales)

In [ ]:
critic_X360_data.critic_score.corr(critic_X360_data.other_sales)

Есть зависимость между оценками критиков и общим количеством проданных копий игры. кореляция составляет 0,29.
если смотреть корреляцию по регионам, то самоя маленькая кореляция в регионе Японии, самая большая в сставшихся регионах(то есть не в северной Америке и не в Европе)

#### отзывы пользователей user_score

In [ ]:
user_X360_data = X360_data[X360_data['user_score'] > 0]
user_X360_data.head(3)

In [ ]:
user_X360_data.plot(x='user_score', y='total_sales', kind='scatter', alpha=0.2, grid=True, ylim=(0,3))
plt.title("отношение количества проданных копий к оценкам пользователей")
plt.ylabel("продажи милионов копий")
plt.xlabel("оценки пользователей")
plt.show()

In [ ]:
user_X360_data.user_score.corr(user_X360_data.total_sales)

In [ ]:
user_X360_data.user_score.corr(user_X360_data.na_sales)

In [ ]:
user_X360_data.user_score.corr(user_X360_data.eu_sales)

In [ ]:
user_X360_data.user_score.corr(user_X360_data.jp_sales)

In [ ]:
user_X360_data.user_score.corr(user_X360_data.other_sales)

Зависимости между количеством проданных игр и оценками пользователей нет, в отличие от зависимости с оценками критиков. И составляет 0.0026. Самая большая корреляция, 0.13 в регионе Японии, в остальных регионах корреляция отсутствует.

### Соотнесите выводы с продажами игр на других платформах.


In [ ]:
PC_data = actual_data[actual_data['platform'] == 'PC']
PC_data_critic = PC_data[PC_data['critic_score'] > 0]
PC_data_user = PC_data[PC_data['user_score'] > 0]

In [ ]:

print(len(actual_data[(actual_data['platform'] == 'PC') & (actual_data['user_score'] > 0)]))
print(len(actual_data[(actual_data['platform'] == 'PC') & (actual_data['critic_score'] > 0)]))
print(len(PC_data))

In [ ]:
PC_data_critic.critic_score.corr(PC_data_critic.total_sales)

In [ ]:
PC_data_user.user_score.corr(PC_data_user.total_sales)

In [ ]:
actual_rating_critic_data = actual_data[actual_data['critic_score'] > 0]
actual_rating_user_data = actual_data[actual_data['user_score'] > 0]

In [ ]:
actual_rating_critic_data.head(3)

In [ ]:
actual_rating_user_data.head(3)

Так спешил в надежде, что смогу продолжить обучение без перерыва, что не проверил работоспособность кода, сейчас проверил все рнаботает, также подправил графики вначале работы.

In [ ]:
sampl_top_actual_platform = top_actual_platform.sample(10)

In [ ]:
sampl_top_actual_platform

In [ ]:
for i in sampl_top_actual_platform:
    actual_rating_critic_data[actual_rating_critic_data['platform'] == i].plot(x='critic_score', y='total_sales', kind='scatter', alpha=0.2, grid=True, ylim=(0,3))
    plt.title(f'влияние оценок критиков на продажи платформы {i}')
    plt.ylabel("продажи милионов копий")
    plt.xlabel("оценки критиков")
    print('у платформы', i, 'корреляция равнв',actual_rating_critic_data[actual_rating_critic_data['platform'] == i].critic_score.corr(actual_rating_critic_data[actual_rating_critic_data['platform'] == i].total_sales))
    print(f'количество игр проданых платформой {i} за актуальгое время равно {len(actual_rating_critic_data[actual_rating_critic_data["platform"] == i])}')

In [ ]:
for i in sampl_top_actual_platform:
    actual_rating_user_data[actual_rating_user_data['platform'] == i].plot(x='user_score', y='total_sales', kind='scatter', alpha=0.2, grid=True, ylim=(0,3))
    plt.title(f'влияние оценок пользователен на продажи платформы {i}')
    plt.ylabel("продажи милионов копий")
    plt.xlabel("оценки пользователей")
    print('у платформы', i, 'корреляция равнв',actual_rating_user_data[actual_rating_user_data['platform'] == i].user_score.corr(actual_rating_user_data[actual_rating_user_data['platform'] == i].total_sales))
    print(f'количество игр проданых платформой {i} за актуальгое время равно {len(actual_rating_user_data[actual_rating_user_data["platform"] == i])}')

закономерносте между рейтингом игры у пользователей и количеством продаж не выявлено. У некоторых платформ небольшая положительная корреляция, у некоторых небольшая отрицательная корреляция, но чаще всего корреляции не просматривается. 

### Посмотрите на общее распределение игр по жанрам. 
Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?

In [ ]:
actual_data.pivot_table(index='genre',  values='total_sales', aggfunc='median').sort_values(by='total_sales', ascending=False).\
plot(kind='bar', grid=True, figsize=(15, 10) )
plt.title("отношение количества проданных копий к жанру игры")
plt.xlabel("Жанр игры")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
top_actual_data.pivot_table(index='genre',  values='total_sales', aggfunc='median').sort_values(by='total_sales', ascending=False).\
plot(kind='bar', grid=True, figsize=(15, 10) )
plt.title("отношение количества проданных игр на топовых актуальных платформах  к жанру игры")
plt.xlabel("Жанр игры")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
X360_data.pivot_table(index='genre',  values='total_sales', aggfunc='median').sort_values(by='total_sales', ascending=False).\
plot(kind='bar', grid=True, figsize=(15, 10) )
plt.title("отношение количества проданных игр на платформе X360 к жанру игры")
plt.xlabel("Жанр игры")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
X360_data.pivot_table(index='genre',  values='total_sales', aggfunc='median').sort_values(by='total_sales', ascending=False)

In [ ]:
PS3_oll_data = actual_data[actual_data['platform'] == 'PS3']
PS3_oll_data.pivot_table(index='genre',  values='total_sales', aggfunc='median').sort_values(by='total_sales', ascending=False).\
plot(kind='bar', grid=True, figsize=(15, 10) )
plt.title("отношение количества проданных игр на платформе PS2 к жанру игры")
plt.xlabel("Жанр игры")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
PS3_oll_data.pivot_table(index='genre',  values='total_sales', aggfunc='median').sort_values(by='total_sales', ascending=False)

In [ ]:
actual_data.pivot_table(columns='genre', index='name' , values='total_sales', aggfunc='median').plot(kind='box', grid=True, figsize=(15, 10) )
plt.title("распределение медианных значений продаж игр по жанрам")
plt.xlabel("Жанр игры")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
actual_data.pivot_table(columns='genre', index='name' , values='total_sales', aggfunc='median').plot(kind='box', grid=True, figsize=(15, 10), ylim=(0, 4) )
plt.title("распределение медианных значений продаж игр по жанрам")
plt.xlabel("Жанр игры")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
actual_data.pivot_table(index='genre',  values='total_sales', aggfunc='median').sort_values(by='total_sales', ascending=False)

При анализе игр по жанрам модно сказать, что наибольшее медианное значение продаж на всех платформах у игр жанра Shooter и оно равно 0.450 (с большим отрывом), затем идет Sports с 0.240 и Platform с 0.225 неньше всего медиана у Strategy с 0.080, Puzzle и Adventure. Также видно, что у разных платформ рейтинг медианных продаж разный. например у PS3 он практически не отличается от данных по всем платформам и составляют Shooter	0.410, Sports 0.260 и Platform	0.25, а у X360 медианные значения топ 3 продаж составляют Role-Playing 0.490 и Sports 0.430 больше чем у Shooter 0.365

В зависимости от платформы или времени лидер в категории жанры может поменятся, но в топ 3 жанра всегда выбиваются жанры Action, Sports и Shooter. а в аутсайдерах остаются Strategy, Adventure и Puzzle. Жанры Racing, Misc, Role-Playing, чаще всего занимают с 4 по 6е места по популярности. А жанры Fighting, Platform, Simulation занимают места с 7го по 9е места.

In [ ]:
data.genre.unique()

## Шаг 4. Составьте портрет пользователя каждого региона


### Определите для пользователя каждого региона (NA, EU, JP): Самые популярные платформы (топ-5). 
Опишите различия в долях продаж.

In [ ]:
top_5_na_data = actual_data.groupby('platform')['na_sales'].sum().sort_values(ascending=False).head(5)
top_5_na_data

In [ ]:
top_5_na_data.plot(kind='bar', grid=True)
plt.title("Самые популярные платформы на Американском регионе")
plt.xlabel("платформа")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
top_5_eu_data = actual_data.groupby('platform')['eu_sales'].sum().sort_values(ascending=False).head(5)
top_5_eu_data

In [ ]:
top_5_eu_data.plot(kind='bar', grid=True)
plt.title("Самые популярные платформы в Европейском регионе")
plt.xlabel("платформа")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
top_5_jp_data = actual_data.groupby('platform')['jp_sales'].sum().sort_values(ascending=False).head(5)
top_5_jp_data

In [ ]:
top_5_jp_data.plot(kind='bar', grid=True)
plt.title("Самые популярные платформы в Японсков регионе")
plt.xlabel("платформа")
plt.ylabel("продажи милионов копий")
plt.show()

Если смотреть на данные за актуальное время то можно увидеть, что в америке самая популярная платформа PS4 108.74, затем
XOne 93.12, X360 81.66,PS3 63.50 и замыкает 5ку 3DS 38.20  в Европе самая популярная платформа с большим отрывом PS4 141.09, затем идут PS3 67.81, XOne 51.59, X360 42.52 и замыкает 5ку 3DS 30.96. В свою очередь, в Японии самая популярная платформа 3DS    67.81, PS3 23.35, PSV 18.59, PS4 15.96, WiiU 10.88. 

### Определите для пользователя каждого региона (NA, EU, JP): Самые популярные жанры (топ-5). 
Поясните разницу.

In [ ]:
actual_data.head(3)

In [ ]:
region_data = actual_data
region_data = region_data.rename(columns={'na_sales':'Американского','eu_sales':'Еевропейского','jp_sales':'Японского'})
region_data
region =['Американского','Еевропейского','Японского']

plt.figure(figsize=[15, 15])
namber = 1
for i in region:
    x_data = region_data.copy()    
    x_top = region_data.groupby('genre')[i].sum().sort_values(ascending=False).reset_index().genre.head(5)
    
    x_data.loc[~x_data['genre'].isin(x_top), 'genre'] = 'другой'
    print(f"статистика жанров {i} региона \n {x_data.groupby('genre')[i].sum().sort_values(ascending=False)}")
    print(('-')*30)  
    plt.subplot(1, 3, namber)    
    x_data.groupby('genre')[i].sum().plot(kind='pie', grid=True, figsize=(15,15), autopct='%1.0f%%')
    plt.title(f'диаграмма жанров {i} региона')
 
    plt.ylabel('')
    
    namber +=1   


В Европейском как и в Американском регионе доли продаж игр по жанрам распределены следующим образом: Action 30%, Shooter         23%, Sports 15%, Role-Playing 10%, Misc(в Америке) и Racing(в Европе)  6% и оставшиеся игры 16%. В то время как в Японсом регионе доли рынка поделены следующим образом: Role-Playing 36%, Action 29%, Misc 7%, Fighting 5%, Shooter 5% и оставшиеся жанры 18%

In [ ]:
actual_top_5_na_genre = actual_data.groupby('genre')['na_sales'].sum().sort_values(ascending=False).head(5)
actual_top_5_na_genre

In [ ]:
actual_top_5_na_genre.plot(kind='bar', grid=True)
plt.title("Актуальные жанры на Американском регионе")
plt.xlabel("жанры")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
actual_top_5_eu_genre = actual_data.groupby('genre')['eu_sales'].sum().sort_values(ascending=False).head(5)
actual_top_5_eu_genre

In [ ]:
actual_top_5_eu_genre.plot(kind='bar', grid=True)
plt.title("Актуальные жанры В Европейском регионе")
plt.xlabel("жанры")
plt.ylabel("продажи милионов копий")
plt.show()

In [ ]:
actual_top_5_jp_genre = actual_data.groupby('genre')['jp_sales'].sum().sort_values(ascending=False).head(5)
actual_top_5_jp_genre

In [ ]:
actual_top_5_jp_genre.plot(kind='bar', grid=True)
plt.title("Актуальные жанры В Японском регионе")
plt.xlabel("жанры")
plt.ylabel("продажи милионов копий")
plt.show()

В актуальное время популярность жанров в Европе не кардинально не изменилаись, за исключением поменялись местами жанры Misc и Racing. Американский рынок также затронули небольшие изменения из топ 5 пропал жанр Platform, жанр Misc занял 4е место и поднял в популярности и вышел на 5е место жанр Role-Playing. В японском регионе жанры изменились кардинально. на 1м месте Role-Playing, на ним идут Action, Misc, Sports и заканчивает 5ку жанр Adventure. Эта разница скорее всего возникла из за того, что в Японском регионе преобладают другие платформы, возможно, что со временем в этом регионе игровая культура пошла развиватся в другую сторону. 

### Определите для пользователя каждого региона (NA, EU, JP): Влияет ли рейтинг ESRB на продажи в отдельном регионе?

In [ ]:
data.head(3)

In [ ]:
rating_data = actual_data
rating_data.head(3)

In [ ]:
rating_data.pivot_table(index='rating', values='na_sales', aggfunc='sum').\
plot(kind='pie', subplots=True, figsize=(10,10), autopct='%1.0f%%')
plt.title("Отношение продаж к рейтингу игры в Североамериканском регионе")
plt.ylabel("рейтинг")
plt.show()

In [ ]:
rating_data.pivot_table(index='rating', values='eu_sales', aggfunc='sum').\
plot(kind='pie', subplots=True, figsize=(10,10), autopct='%1.0f%%')
plt.title("Отношение продаж к рейтингу игры в Европейском регионе")
plt.ylabel("рейтинг")

In [ ]:
rating_data.pivot_table(index='rating', values='jp_sales', aggfunc='sum')\
.plot(kind='pie', subplots=True, figsize=(10,10), autopct='%1.0f%%')
plt.title("Отношение продаж к рейтингу игры в Японском регионе")
plt.ylabel("рейтинг")
plt.show()

In [ ]:
rating_data.pivot_table(index='rating', values='other_sales', aggfunc='sum')\
.plot(kind='pie', subplots=True, figsize=(10,10), autopct='%1.0f%%')
plt.title("Отношение продаж к рейтингу игры в Остальных регионах")
plt.ylabel("рейтинг")
plt.show()

In [ ]:
rating_data.pivot_table(index='rating', values=('na_sales','eu_sales','jp_sales','other_sales'), aggfunc=('sum'))

In [ ]:
rating_data.pivot_table(index='rating', values=('na_sales','eu_sales','jp_sales','other_sales'), aggfunc=('sum')).plot(kind='bar', grid=True, figsize=(15, 10))
plt.title("Актуальные жанры В Японском регионе")
plt.xlabel("жанры")
plt.ylabel("продажи милионов копий")
plt.show()

Доля проданных игр относительно категории игры в во всех регионах, за исключением Японского, примерно равнва и составляют следующте показатели: "Е" в раене 19%, "Т" в раене 11%, "М" в раене 39% и "Е10+" в раене 11% игре не отмеченые рейтингом составляют 20% в то время как в Японском региионе игр с рейтингом "Е"составляют 11% и "Т" составляет 15%, игры из рейтинга "М" составляет 10% и "Е10+" составляет 4%, а игры без рейтинга составляют 60%

## Шаг 5. Проверьте гипотезы


### Гипотеза 1 Средние пользовательские рейтинги платформ Xbox One и PC одинаковые;




Задайте самостоятельно пороговое значение alpha.
Поясните:
Как вы сформулировали нулевую и альтернативную гипотезы;
Какой критерий применили для проверки гипотез и почему.

Для проверки гипотез будемприменять метод st.ttest_ind с уровнем значимости в 5%. Так как у нас проверка средних у 2х генеральных совокупностей. Предположим, что средние этих совокупностей равны. были выдвынуты следующие гипотезы:

H0: Средние пользавательские рейтинги платформы Xbox One равны средних пользовательских рейтингов платформы PC.

H1: Средние пользавательские рейтинги платформы Xbox One не равны средним пользовательским рейтингам платформы PC. 

In [ ]:
XOne_data = actual_data[(actual_data['platform'] == 'XOne') & (actual_data['user_score'] > 0)].user_score
PC_data = actual_data[(actual_data['platform'] == 'PC') & (actual_data['user_score'] > 0)].user_score
len(XOne_data)

In [ ]:
len(PC_data)

In [ ]:
XOne_data = XOne_data.sample(157)
len(PC_data)

In [ ]:
sample_1 = XOne_data
sample_2 = PC_data

alpha = 0.05
results = st.ttest_ind(sample_1, sample_2, alternative='two-sided')

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу') 

In [ ]:
actual_data[(actual_data['platform'] == 'XOne') & (actual_data['user_score'] > 0)].user_score.mean()

In [ ]:
actual_data[(actual_data['platform'] == 'PC') & (actual_data['user_score'] > 0)].user_score.mean()

Вывод: Не получилось отвергнуть нулевую гипотезу. Средние пользавательские рейтинги платформы Xbox One равны средним пользовательским рейтингов платформы PC.

### Гипотеза 2 Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.
Задайте самостоятельно пороговое значение alpha.
Поясните:
Как вы сформулировали нулевую и альтернативную гипотезы;
Какой критерий применили для проверки гипотез и почему.


Для проверки гипотез будемприменять метод st.ttest_ind с уровнем значимости в 5%. Так как у нас проверка средних у 2х генеральных совокупностей. Предположим, что средние этих совокупностей равны. были выдвынуты следующие гипотезы:

H0: Средние пользовательские рейтинги жанра Action равны средним пользовательским рейтингам жанра Sports.

H1: Средние пользовательские рейтинги жанра Action не равны средним пользовательским рейтингам жанра Sports.

In [ ]:
action_data = actual_data[(actual_data['genre'] == 'Action') & (actual_data['user_score'] > 0)].user_score
sports_data = actual_data[(actual_data['genre'] == 'Sports') & (actual_data['user_score'] > 0)].user_score
len(action_data)

In [ ]:
len(sports_data)

In [ ]:
action_data = action_data.sample(167)

In [ ]:
sample_1 = action_data
sample_2 = sports_data

alpha = 0.05
results = st.ttest_ind(sample_1, sample_2, alternative='two-sided' )

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу') 

In [ ]:
actual_data[(actual_data['genre'] == 'Action') & (actual_data['user_score'] > 0)].user_score.mean()

In [ ]:
actual_data[(actual_data['genre'] == 'Sports') & (actual_data['user_score'] > 0)].user_score.mean()

Вывод: Отвергаем нулевую гипотезу. H0: Средние пользовательские рейтинги жанра Action не равны средним пользовательским рейтингам жанра Sports.

## Шаг 6. Общий вывод

В таблице нет явных дублекатов, но есть пропуски в столбцах "Name" и "Genre" по 2 пропуска, в столбце "Year_of_Release" 269 пропусков и больше всего пропусков в столбцах: "Critic_Score", "User_Score" и "Rating". Их количество равно: 8578, 6701 и 6766 соответственно. В именах столббцов присутствуют заглавные буквы. Столбци "Year_of_Release" и "User_Score" не соответствуют своему типу данных.

Удалены 2 строки, у которой почти все данные заполнены NaN.Так как считаю эти данные ошибкой не влияющей на исследование.

в столбце "critic_score" пропуски частично заполннены средними значениями из их таких же игр, но с других платформ, а оставшиеся пропуски заменены на значение "-99999" и тип данных столбца изменен на "int"

в строке "user_score" пропущеные значения били частично заменены на средние значения игры на других платформах, если таких игр не быо,то значения были изменены на значение "-99999", также были найдены 2424 значения "tbd". Преддполагаю, что это значение ошибочное, и так как в столбце "user_score" дискретные математические значения, мной было принято решение заменить значение "tbd" на значение "-88888". А также произведя матиматические действия измененить тип столбца на тип "int", для возможности использования данных в арифмитеческих выражениях.

Не заполнял эти данные не мидианными, не средними значениями, так как от этих данным может меняться мнение об игре. Недостающие данные в столбце "rating" заменены на значение "no_rating", предполагаю, что в эти игры можно играть всем, но в расчетах по рейтингу эти данные использоватся не будут.

Причин появления пропусков в данных может быть много, скорее всего они связаны с тем, что в открытый доступ часто поступают не полные данные.

В наших данных есть записи с 1980 по 2016 год. Больше всего выпущено игр в 2008 год что составляет 8.5%. Меньше всего в 1980м, всего 9 игр что составляет 0.053% Так как до 1993г выпускалось мало игр, моожно считать, что эти данные не актуальны. (выбросы). Количество игр выпущенное с 2000 до 2008 года равно 7417 игр, что составляет 45%, в то время как после 2008 года выпущено 6703 игры, что составляет 40 процентов от всех выпущеных игр.

Платформы с наибольшими продажами, топ 7 платформ, занимают 66% всего рынка видеоигр. Актуальный срок жизни платформы составляет 10 - 12 лет.

Было принято решение считать актуальным, то время в которое выходят игры на современные платформф, так как жижнь платформы примерна равна 10 годам, то и актуальнам будут данные за последние 10 лет. также найдем топ 10 платформ за последние 10 лет. Самой выгодной платформой за актуальное время является PS3 она подала 910.38 милионов копий игр, затем идет X360, с продажами в 901.37 копий, потом Wii 754.03 и DS с продажами в 535.54 милиона копий. PS4 с продажами 314.14, 3DS с продажами 257.81, PSP с продажами 183.31, PS2 с продажами 162.38, PC с продажами	160.57 и замыкает 10ку XOne с продажами	159.32

Из десяти платформ, которые за последние 10 лет продами максимальгое количество копий только 2 платформы показывают потенциальный рост: это платформы XOne и PS4. На них стоит обратить особое внимание. четыре платформы в 2016 году вообще не показали прибыли или очень маленькую прибыль: PSP, PSV, PC и Wii про них стоит забыть, их время ушло. И четыре платформы, которые скоро уйдут, но еще могут принести некоторцю прибыль. это 3DS, WiiU, PS3 и X360.

график "ящик с усами" не показывают картину в целом для платформ по глобальным продажам. И в первую очередь это связано с тем, что "ящик с усами" считает многие проданые игры выбросами, в то время как эти игры принесли платформе основной доход, Максимальная медиана проданых копий принадлежит платформе X360 и составляет 0.265 милиона копий, а доход учитывается со 186 игрр по которым есть данные, в то время как лидер продаж платворма PS4 имеет медиану проданых копий равную 0.2 милоинам и включает данные по 392 проданым играм. Но также мы можем увидеть, что чем жирнее хвостда графике с усами, тем больше продаж у этой платформы.

При исследовании влияния на продажи внутри одной популярной платформы отзывов пользователей и критиков. Я решил проводить анализ платформы X360. так как это популярная платформа входит в топ 10 и уже прошло ее время, значит по ней представлены данные за весь период жизни платформы.

Есть зависимость между оценками критиков и общим количеством проданных копий игры. кореляция составляет 0,29.
если смотреть корреляцию по регионам, то самоя маленькая кореляция в регионе Японии, самая большая в сставшихся регионах(то есть не в северной Америке и не в Европе)

Зависимости между количеством проданных игр и оценками пользователей нет, в отличие от зависимости с оценками критиков. И составляет 0.0026. Самая большая корреляция, 0.13 в регионе Японии, в остальных регионах корреляция отсутствует.

Если смотреть по другим платформам, то просматривается закономерность между отзывами критиков и количством проданных игр в платфлрах, при условии, что этих игр было продано более 100 едениц. в этих случаях корреляция положительная и более 0.26, за исключением платформы PC, у которой корреляция равна 0.19. Самая большая корреляция у платформы PS4, она равна 0.39 и платформы XOne, она равна 0.37

Но закономерностей между рейтингом игры у пользователей и количеством продаж не выявлено. У некоторых платформ небольшая положительная корреляция, у некоторых небольшая отрицательная корреляция, но чаще всего корреляции не просматривается.

При анализе игр по жанрам модно сказать, что наибольшее медианное значение продаж на всех платформах у игр жанра Shooter и оно равно 0.450 (с большим отрывом), затем идет Sports с 0.240 и Platform с 0.225 неньше всего медиана у Strategy с 0.080, Puzzle и Adventure. Также видно, что у разных платформ рейтинг медианных продаж разный. например у PS3 он практически не отличается от данных по всем платформам и составляют Shooter	0.410, Sports 0.260 и Platform	0.25, а у X360 медианные значения топ 3 продаж составляют Role-Playing 0.490 и Sports 0.430 больше чем у Shooter 0.365

Если смотреть на данные за актуальное время то можно увидеть, что в америке самая популярная платформа PS4 108.74, затем
XOne 93.12, X360 81.66,PS3 63.50 и замыкает 5ку 3DS 38.20  в Европе самая популярная платформа с большим отрывом PS4 141.09, затем идут PS3 67.81, XOne 51.59, X360 42.52 и замыкает 5ку 3DS 30.96. В свою очередь, в Японии самая популярная платформа 3DS    67.81, PS3 23.35, PSV 18.59, PS4 15.96, WiiU 10.88. 

В Европейском как и в Американском регионе доли продаж игр по жанрам распределены следующим образом: Action 30%, Shooter         23%, Sports 15%, Role-Playing 10%, Misc(в Америке) и Racing(в Европе)  6% и оставшиеся игры 16%. В то время как в Японсом регионе доли рынка поделены следующим образом: Role-Playing 36%, Action 29%, Misc 7%, Fighting 5%, Shooter 5% и оставшиеся жанры 18%

Самыми популярными за все время в Европейском и Американском регионе являются следующие жанры: Action, Sports и Shooter, на 4м месте в Америке находится жанр Platform, а в Европе Racing. 5ю строчку в обоих регионах занимают жанр Misc.

Доля проданных игр относительно категории игры в во всех регионах, за исключением Японского, примерно равнва и составляют следующте показатели: "Е" в раене 19%, "Т" в раене 11%, "М" в раене 39% и "Е10+" в раене 11% игре не отмеченые рейтингом составляют 20% в то время как в Японском региионе игр с рейтингом "Е"составляют 11% и "Т" составляет 15%, игры из рейтинга "М" составляет 10% и "Е10+" составляет 4%, а игры без рейтинга составляют 60%

Также были проверены 2 гипотезы:

Гипотеза 1 Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.

Так как рейтинги не могут быть абсолютно одинаковыми были выдвынуты следующие гипотезы:

H0: Средние пользавательские рейтинги платформы Xbox One равны средних пользовательских рейтингов платформы PC.

H1: Средние пользавательские рейтинги платформы Xbox One не равны средним пользовательским рейтингам платформы PC.

Применялся метод st.ttest_ind( alternative='greater' )

Вывод: Не получилось отвергнуть нулевую гипотезу. Средние пользавательские рейтинги платформы Xbox One равны средним пользовательским рейтингов платформы PC.

Гипотеза 2 Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

Рейтинги у жанров врядли будут одинаковыми, поэтому мы выделяем один из них над другим.

H0: Средние пользовательские рейтинги жанра Action меньше средних пользовательских рейтингов жанра Sports.

H1: Средние пользовательские рейтинги жанра Action больше либо равны равны средним пользовательским рейтингам жанра Sports.

Был использован метод st.ttest_ind( alternative='less' )

Вывод: Не получилось отвергнуть нулевую гипотезу. H0: Средние пользовательские рейтинги жанра Action меньше средних пользовательских рейтингов жанра Sports.

Подводя итоги можно предположить, что стоит обратить особое внимание на 2 платформы показывают потенциальный рост: это платформы XOne и PS4. Наиболее выгодными жанрами будут Shooter, Action, Sports для Североамериканского и Европейскогго региона и Role-Playing или Action для Японского региона. Также игры с рейтингом "М" будут выгодно продаваться во всех регионах.